In [0]:
# pyspark functions
from pyspark.sql.functions import *
# URL processing
import urllib
# Define the path to the Delta table
delta_table_path = "dbfs:/user/hive/warehouse/authentication_credentials"

# Read the Delta table to a Spark DataFrame
aws_keys_df = spark.read.format("delta").load(delta_table_path)

# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.select('Secret access key').collect()[0]['Secret access key']
# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")
# AWS S3 bucket name
AWS_S3_BUCKET = "user-0a1e5630c127-bucket"
# Mount name for the bucket
MOUNT_NAME = "/mnt/0a1e5630c127"
# Source url
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

Out[21]: True

In [0]:
display(dbutils.fs.ls("/mnt/0a1e5630c127"))

path,name,size,modificationTime
dbfs:/mnt/0a1e5630c127/kafka-connect-s3/,kafka-connect-s3/,0,1704726356899
dbfs:/mnt/0a1e5630c127/topics/,topics/,0,1704726356899


In [0]:
%sql
SET spark.databricks.delta.formatCheck.enabled=false

key,value
spark.databricks.delta.formatCheck.enabled,false


In [0]:
# File location and type
# Asterisk(*) indicates reading all the content of the specified file that have .json extension
file_location ="dbfs:/mnt/0a1e5630c127/topics/0a1e5630c127.geo/partition=0/*.json"
file_type = "json"
# Ask Spark to infer the schema
infer_schema = "true"
# Read in JSONs from mounted S3 bucket
df = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location)
# Display Spark dataframe to check its content
display(df)

country,ind,latitude,longitude,timestamp
Cocos (Keeling) Islands,10794,-89.5236,-154.567,2022-01-01 02:26:50
French Guiana,8304,-28.8852,-164.87,2019-09-13 04:50:29
Cote d'Ivoire,4315,-45.8508,66.1003,2019-12-15 03:51:28
Albania,7528,-89.9787,-173.293,2020-08-28 03:52:47
Armenia,2863,-5.34445,-177.924,2020-04-27 13:34:16
Colombia,5730,-77.015,-101.437,2021-04-19 17:37:03
Maldives,1313,77.0447,61.9119,2018-06-26 02:39:25
Aruba,8731,-83.104,-171.302,2020-07-17 04:39:09
